In [1]:
#importing all the dependencies packages
import json
import csv
import nltk
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem     import PorterStemmer
from nltk.stem     import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import string
import re
import csv
import pandas as pd
import numpy as np
st = LancasterStemmer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))    
def extract_documents(sentence1,productName):
    #removing punctuations and stopwords to extract main relavant data
    document_list = []
    sentence1 = sentence1.lower() # lowercase text
    sentence1 = REPLACE_BY_SPACE_RE.sub(' ', sentence1) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    sentence1 = BAD_SYMBOLS_RE.sub('', sentence1) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    sentence1 = sentence1.replace('x', '')
    sentence1 = ' '.join(word for word in sentence1.split() if word not in STOPWORDS) # remove stopwords from text
    extracted_words_from_sentence = set()
    
    for key in end_json:
        for word in end_json[key]:
            pattern = r"\b" + re.escape(word) + r"\b"
            match = re.search(pattern, sentence1)
            if match:
                extracted_words_from_sentence.add(key)
#     print("words from sentence")
#     print(extracted_words_from_sentence)
    #comparisions based on products
    df = pd.read_excel (r'./test_sample_1.xlsx')
    data = df[['Keywords (m)','Product','Document Name','DOC ID']]
    data = data.sort_values(by=['Product'])
    data = data.dropna()
    data = data.replace(r'^\s*$', np.nan, regex=True)
    data['Product'] = data['Product'].apply(str.lower)
    data['extracted_words'] = str(extracted_words_from_sentence)
    data['Keywords (m)'] = data['Keywords (m)'].str.lower()
    data = data[data.Product.str.contains('none|n/a|no data|nan')==False]
    data = data[data.Product.str.contains(str(productName).lower())]
    data['new_keywords'] = ''
    data['score']=0.0
    #importing the keywords excel for comparision
    for sent in data['Keywords (m)']:
        t = re.sub(r'\s*,\s*',',',str(sent))
        data['Keywords (m)'] = data['Keywords (m)'].replace([sent],t)
    for idx1,sentence2 in data['Keywords (m)'].iteritems():
        string_words_new = ''
        extracted_words_from_key_words = set()
        if str(sentence2).lower() not in ('nan','n/a','none','no data'):
            for key in end_json:
                for word in end_json[key]:
                    new_sentence_split = np.array(sentence2.split(','))
                    if word in new_sentence_split:
                        extracted_words_from_key_words.add(key)
            for val in extracted_words_from_key_words:
                string_words_new = string_words_new+ ',' + val
            data.at[idx1, 'new_keywords'] = string_words_new
    for idx,sentence in data['new_keywords'].iteritems():
        if str(sentence).lower() not in ('nan','n/a','none','no data','','[none]'):
            array_toCompare = np.array(sentence.split(','))
            flt_sent2 = [w for w in array_toCompare if not w.lower() in STOPWORDS and w not in string.punctuation and w not in ('nan','n/a','none','no data','','[none]')]
            ac = [x.lower() for x in flt_sent2]
            common_elements = list(extracted_words_from_sentence.intersection(ac))
            union_elements = list(set(extracted_words_from_sentence).union(set(ac)))
            #Calculating the score
            score = len(common_elements)/len(union_elements)*100
            data.at[idx, 'score'] = score
    data.to_csv('pavan.csv')
    df_final = data.nlargest(3, 'score')
    result_final_output = df_final.to_json(orient="records")
    #result_final_json_output = json.loads(result_final_output)
    return result_final_output

In [31]:
def remove_stopwords_spacy(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#spacy lemmatize
def lemmatize_text_spacy(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return lemmatized_words
#spacy Stemming
def stem_text_spacy(text):
    stemmed_words = [Word(word).stem() for word in text]
    stemmed_text = " ".join(stemmed_words)
    return stemmed_text

def count_matching_words(sentence, words):
    sentence_words = sentence.lower()
    matching_words = [word for word in words if word.lower() in sentence_words]
    return len(matching_words)

def replace_words_with_dict(sentence,json_data):
    modified_sentence = sentence.lower()
    for key, word_array in end_json.items():
        for word in word_array:
            modified_sentence = modified_sentence.replace(word, key)  # Replace word with its corresponding value
    return modified_sentence

In [32]:
#new trail
#importing all the dependencies packages
import json
import csv
import nltk
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem     import PorterStemmer
from nltk.stem     import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import string
import re
import csv
import pandas as pd
import numpy as np
import spacy
import en_core_web_sm
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer
nlp = spacy.load("en_core_web_sm")
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english')) 
# st = LancasterStemmer()   
def extract_documents(sentence1,productName):
    #removing punctuations and stopwords to extract main relavant data
    json_data = end_json
    document_list = []
    sentence1 = sentence1.lower() # lowercase text
    sentence1 = REPLACE_BY_SPACE_RE.sub(' ', sentence1) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    sentence1 = BAD_SYMBOLS_RE.sub('', sentence1) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    sentence1 = sentence1.replace('x', '')
    sentence1 = remove_stopwords_spacy(sentence1)
    stem_words_text = replace_words_with_dict(sentence1,json_data) # looping through json for synonyms and abbreviations
    lemma_sentence = lemmatize_text_spacy(stem_words_text) #apply lemmatization for sentence
    stem_text = stem_text_spacy(lemma_sentence) #apply stemming for sentence
    print(stem_text)
    #comparisions based on products
    df = pd.read_excel (r'./test_sample_1.xlsx')
    data = df[['Keywords (m)','Product','Document Name','DOC ID']]
    data = data.sort_values(by=['Product'])
    data = data.dropna()
    data = data.replace(r'^\s*$', np.nan, regex=True)
    data['Product'] = data['Product'].apply(str.lower)
    data['extracted_sentence'] = str(stem_text)
    data['Keywords (m)'] = data['Keywords (m)'].str.lower()
    data = data[data.Product.str.contains('none|n/a|no data|nan')==False]
    data = data[data.Product.str.contains(str(productName).lower())]
    data['new_keywords'] = ''
    data['score']=0.0
    #importing the keywords excel for comparision
    for sent in data['Keywords (m)']:
        t = re.sub(r'\s*,\s*',',',str(sent))
        data['Keywords (m)'] = data['Keywords (m)'].replace([sent],t)
    for idx1,sentence2 in data['Keywords (m)'].iteritems():
        string_words_new = ''
        extracted_words_from_key_words = set()
        if str(sentence2).lower() not in ('nan','n/a','none','no data'):
            sentence2 = remove_stopwords_spacy(sentence2) # remove stopwords from text
            stem_words_text = replace_words_with_dict(sentence2,json_data) # looping through json for synonyms and abbreviations
            lemma_words = lemmatize_text_spacy(stem_words_text)
            stem_text2 = stem_text_spacy(lemma_words)
            data.at[idx1, 'new_keywords'] = stem_text2
    for idx,sentence in data['new_keywords'].iteritems():
        if str(sentence).lower() not in ('nan','n/a','none','no data','','[none]'):
            array_toCompare_initial = np.array(sentence.split(','))
            array_toCompare_final = [elem.strip().lower() for elem in array_toCompare_initial]
            flt_sent2 = [w for w in array_toCompare_final if not w.lower() in STOPWORDS and w not in string.punctuation and w not in ('nan','n/a','none','no data','','[none]')]
            flt_sent2_final = list(set(flt_sent2))
            display(flt_sent2_final)
            matching_word_count = count_matching_words(stem_text, flt_sent2_final)
            common_elements = matching_word_count
            #Calculating the score
            score = common_elements/len(flt_sent2_final)*100
            data.at[idx, 'score'] = score
    data.to_csv('pavan.csv')
    df_final = data.nlargest(3, 'score')
    result_final_output = df_final.to_json(orient="records")
    #result_final_json_output = json.loads(result_final_output)
    return result_final_output

In [33]:
print(extract_documents("Is there data that can be emailed to help support prescribing Ozempic for a patient with type 2 diabetes where the patient had an inadequate A1C response with Metformin and insulin. Also, the patient has a family history of cardiovascular disease.","Ozempic"))

datum email help support prescrib ozemp patient type 2 diabet patient inadequ a1c respons metformin insulin    patient famili histori cvd


['excurs']

['niosh', 'hazard']

['guidentificationelin', 'glp-1', 'recommend', 'therapi', 'place']

['excel',
 'glp-1 ra',
 'mace',
 'cardiovascular',
 'outcom',
 'harmoni',
 'elixa',
 'sustain 6',
 'leader',
 'glp-1',
 'cardiovascularot',
 'rewind',
 'pioneer 6']

...

['exposur', 'dose - respons', 'pharmacokinet']

['cardiovasculard',
 'cardiovascular',
 'cardiovascular risk factor',
 'verma',
 'establish cardiovasculard',
 'cardiovascularot',
 'rewind']

['carnitin', 'b12', 'compound']

['diabet', 't2d', 'step', 'sustain', 'fort']

[{"Keywords (m)":"kapitza,beta,insulin,response","Product":"ozempic","Document Name":"Cover to Kapitza et al. (Insulin Response) (SEM 151) (v5.0)","DOC ID":"SEM 151","extracted_sentence":"datum email help support prescrib ozemp patient type 2 diabet patient inadequ a1c respons metformin insulin    patient famili histori cvd","new_keywords":"kapitza , beta , insulin , respons","score":50.0},{"Keywords (m)":"type 1,diabetes,t1dm,t1d","Product":"ozempic","Document Name":"Type 1 Diabetes (SEM 090) (v12.0)","DOC ID":"SEM 090","extracted_sentence":"datum email help support prescrib ozemp patient type 2 diabet patient inadequ a1c respons metformin insulin    patient famili histori cvd","new_keywords":"type 1,diabet , t1d , t1d","score":33.3333333333},{"Keywords (m)":"basal,insulin,glargine,lantus","Product":"ozempic","Document Name":"vs Basal insulin (SEM 113) (v7.0)","DOC ID":"SEM 113","extracted_sentence":"datum email help support prescrib ozemp patient type 2 diabet patient inadequ a1c res

In [17]:
#to calculate accuracy of data for old code
df_accuracy = pd.read_excel (r'./data_to_test.xlsx')
for idx_acc,sentence_acc in df_accuracy['Question'].iteritems() :
    df_accuracy.at[idx_acc, 'accuracy'] = 'not accurate'
    output_sentence = extract_documents(sentence_acc,df_accuracy.at[idx_acc,'Product'])
    json_data = json.loads(output_sentence)
    count = 1
    for dj in json_data:
        if dj['DOC ID'] == df_accuracy.at[idx_acc,'DOC ID']:
            df_accuracy.at[idx_acc, 'accuracy'] = 'accurate with one of docID'
        df_accuracy.at[idx_acc, 'extracted_sentence_'+ str(count)] = str(dj['extracted_words'])
        df_accuracy.at[idx_acc, 'keywords_'+ str(count)] = dj['Keywords (m)']
        df_accuracy.at[idx_acc, 'Document_Name_'+ str(count)] = dj['Document Name']
        df_accuracy.at[idx_acc, 'DOC_ID_'+ str(count)] = dj['DOC ID']
        df_accuracy.at[idx_acc, 'new_keywords_'+ str(count)] = dj['new_keywords']
        df_accuracy.at[idx_acc, 'score_'+ str(count)] = dj['score']
        count+=1
df_accuracy.to_csv('accuracy_old.csv')

['excurs']

['anti - obes medic', 'bariatr surgeri', 'saxenda', 'wegovi']

['glp-1 ra', 'wegovi', 'aom', 'saxenda', 'man 2', 'anti - obes', 'mtc']

['ovari', 'pco', 'polycyst']

['carnitin', 'b12', 'compound']

...

['titrat', 'escal', 'initi', 'dose', 'miss']

['pregnanc',
 'pregnant',
 'breastfe',
 'teratogen',
 'breastf',
 'lactat',
 'fertil',
 'fetu',
 'contracept',
 'fetal']

['obes', 'phase 3', 'investig', 'select', 'step', 'overweight']

['- use',
 'warm',
 'storag',
 'heat',
 'freez',
 'stabl',
 'room',
 'temp',
 'refriger',
 'expir',
 'store',
 'cool',
 'hot',
 'fridentificationg',
 'temperatur',
 'frozen',
 'stabil']

KeyboardInterrupt: 

In [27]:
#to calculate accuracy of data
df_accuracy = pd.read_excel (r'./data_to_test.xlsx')
for idx_acc,sentence_acc in df_accuracy['Question'].iteritems() :
    df_accuracy.at[idx_acc, 'accuracy'] = 'not accurate'
    output_sentence = extract_documents(sentence_acc,df_accuracy.at[idx_acc,'Product'])
    json_data = json.loads(output_sentence)
    count = 1
    for dj in json_data:
        if dj['DOC ID'] == df_accuracy.at[idx_acc,'DOC ID']:
            df_accuracy.at[idx_acc, 'accuracy'] = 'accurate with one of docID'
        df_accuracy.at[idx_acc, 'extracted_sentence_'+ str(count)] = str(dj['extracted_sentence'])
        df_accuracy.at[idx_acc, 'keywords_'+ str(count)] = dj['Keywords (m)']
        df_accuracy.at[idx_acc, 'Document_Name_'+ str(count)] = dj['Document Name']
        df_accuracy.at[idx_acc, 'DOC_ID_'+ str(count)] = dj['DOC ID']
        df_accuracy.at[idx_acc, 'new_keywords_'+ str(count)] = dj['new_keywords']
        df_accuracy.at[idx_acc, 'score_'+ str(count)] = dj['score']
        count+=1
df_accuracy.to_csv('accuracy_new.csv')

['excurs']

['anti - obes medic', 'bariatr surgeri', 'saxenda', 'wegovi']

['glp-1 ra', 'wegovi', 'aom', 'saxenda', 'man 2', 'anti - obes', 'mtc']

['ovari', 'pco', 'polycyst']

['carnitin', 'b12', 'compound']

...

['pen', 'demo', 'demonstr']

['clarif', 'addit inform requir', 'credenti', 'inform', 'clarifi', 'hcp']

['studi particip', 'trial particip']

['report', 'advers event', 'fda']

['guidentificationelin',
 'nafld',
 'sema - nash',
 'nafd / nash',
 'nash',
 'aac',
 'fibrosi',
 'aga']

In [23]:
df_tocheck = pd.read_csv(r'./accuracy_new.csv')
df_tocheck['accuracy'].value_counts()['accurate with one of docID']
df_tocheck['accuracy'].count()
accuracy = df_tocheck['accuracy'].value_counts()['accurate with one of docID']/df_tocheck['accuracy'].count()
print(accuracy)

0.30303030303030304


In [26]:
end_json = {
    "international organization for standardization": [
        "international organization for standardization","iofs"
    ],
    "ibs": ["ibs","irritable bowel syndrome"],
    "xray": ["xray","x-ray","x-rays","xrays"],
    "anti-liraglutide": ["anti-liraglutide","anti liraglutide"],
    "pharmacokinetics": ["pharmacokinetics","pk"],
    "gastrointestinal":["gastrointestinal","gi"],
    "glp-1": ["glp-1","glp 1","glp1"],
    "obesity-related surgery": ["obesity-related surgery","obesity related surgery"],
    "beta cell": ["beta cell","beta-cell"],
    "homa-b": ["homa-b","homab","homa b"],
    "c-peptide": ["c-peptide","c peptide"],
    "homa ir": ["homa ir","homa-ir","homair"],
    "blood brain barrier": ["blood brain barrier","bbb","blood-brain-barrier","blood-brain barrier"],
    "bp": ["bp","blood pressure"],
    "hypertension": ["hypertension","hypertensive"],
    "cardiovascular": ["cardiovascular","cv"],
    "bpm": ["bpm"],
    "high blood pressure": ["high blood pressure","hbp"],
    "systolic blood pressure": ["systolic blood pressure","sbp"],
    "diastolic blood pressure": ["diastolic blood pressure","dbp"],
    "heart rate": ["heart rate","hr"],
    "body mass": ["body mass","body-mass"],
    "type 2": ["type 2","type-2","type2"],
    "arterial": ["arterial","artery","atrial"],
    "cvd":["cardiovascular disease","cvd"],
    "ascvd":["atherosclerotic cardiovascular disease","ascvd"],
    "mace":["major adverse cardiovascular events","mace"],
    "infarction": ["infarction","infarctions"],
    "mi": ["mi","myocardial infarction"],
    "cvot": ["cardiovascular outcome trial","cvot"],
    "hf": ["hf","heart failure"],
    "concomitant": ["concomitant"],
    "oad": ["obstructive airway disease","oad"],
    "lipid lowering": ["lipid lowering","lipid-lowering"],
    "phase 3": ["phase 3","phase3","phase-3"],
    "combination": ["combination","combinations","combine","combo","combining"],
    "long-acting": ["long-acting","long acting"],
    "dpp4": ["dpp4","dpp-4i","dpp-4","dipeptidyl peptidase-4"],
    "dipeptidyl peptidase inhibitor": ["dipeptidyl peptidase inhibitor"],
    "dpp-4i": ["dpp-4i","dpp4i"],
    "anti-obesity": ["anti-obesity","anti obesity"],
    "short-acting": ["short-acting","short acting"],
    "dose-finding": ["dose-finding","dose finding"],
    "dose-ranging": ["dose-ranging","dose ranging"],
    "ozempic 2 mg": ["ozempic 2 mg","ozempic 2mg","ozempic 2"],
    "semaglutide 2mg": ["semaglutide 2mg","semaglutide 2 mg"],
    "pioneer 1": ["pioneer 1","p1"],
    "pioneer 2": ["pioneer 2","p2"],
    "pioneer 3": ["pioneer 3","p3"],
    "pioneer 4": ["pioneer 4","p4"],
    "pioneer 5": ["pioneer 5","p5"],
    "pioneer 6": ["pioneer 6","p6"],
    "pioneer 7": ["pioneer 7","p7"],
    "identification": ["identification","id"],
    "mg": ["mg","milligram","milligrams"],
    "halflife": ["halflife","half-life","half life"],
    "steady-state": ["steady-state","steady state","steadystate"],
    "literature search": ["literature search","lit search","literature  search"],
    "fxa": ["fxa","factor xa"],
    "glp1-ra": ["glp1-ra","glp1ra","glp-1ra"],
    "type 1": ["type 1","type-1","type1","typei","type i","type-i"],
    "t1d": ["t1d","t1dm","tidm","type 1 diabetes"],
    "t1dm":["t1dm","tidm","type 1 diabetes mellitus"]
}


In [9]:
import spacy
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# def lemmatize_text(text):
#     doc = nlp(text)
#     lemmatized_text = " ".join([token.lemma_ for token in doc])
#     return lemmatized_text

# text = "Provider has a patient with type 2 diabetes who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
# lemmatized_text = lemmatize_text(text)
# print(lemmatized_text)
from textblob import Word

def stem_text(text):
    words = text.split()
    stemmed_words = [Word(word).stem() for word in words]
    stemmed_text = " ".join(stemmed_words)
    return stemmed_text

text = "Provider has a patient with type 2 diabetes who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
stemmed_text = stem_text(text)
print(stemmed_text)

provid ha a patient with type 2 diabet who wa diagnos with a benign tumor on the pancreas. she want to find out if we have ani studi that includ patient with the same condition, and is it safe to start them on ozempic?


In [6]:
!pip install textblob

     -------------------------------------- 636.8/636.8 kB 5.0 MB/s eta 0:00:00


In [1]:
import stanza

def stem_text(text):
    stanza.download('en')  # Download the English models

    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')  # Initialize the pipeline

    doc = nlp(text)
    stemmed_text = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
    return stemmed_text

text = "Provider has a patient with type 2 diabetes diabetic who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
stemmed_text = stem_text(text)
print(stemmed_text)

2023-06-06 11:30:23 INFO: Downloading default packages for language: en (English) ...
2023-06-06 11:30:25 INFO: File exists: C:\Users\Owner\stanza_resources\en\default.zip
2023-06-06 11:30:29 INFO: Finished downloading models and saved to C:\Users\Owner\stanza_resources.
2023-06-06 11:30:29 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-06 11:30:29 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-06-06 11:30:30 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2023-06-06 11:30:30 INFO: Using device: cpu
2023-06-06 11:30:30 INFO: Loading: tokenize
2023-06-06 11:30:30 INFO: Loading: pos
2023-06-06 11:30:30 INFO: Loading: lemma
2023-06-06 11:30:30 INFO: Loading: depparse
2023-06-06 11:30:31 INFO: Done loading processors!


provider have a patient with type 2 diabete diabetic who be diagnose with a benign tumor on the pancrea . she want to find out if we have any study that include patient with the same condition , and be it safe to start they on Ozempic ?


In [11]:
!pip install stanza

     -------------------------------------- 802.5/802.5 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 353.7/353.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 172.4/172.4 MB 2.9 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350810 sha256=969d732939157e7f697a04e7abe1e2f89669408633485ac5f8dfc8094f9d5b40
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\d5\c5\45\6c3b2e538c10e9667495d6bff243ee0a296c3c246478007df3
Successfully built emoji


In [8]:
import json
def replace_words_with_dict(sentence,json_data):
#     word_dict = json.loads(end_json)  # Load JSON data into a Python dictionary
    modified_sentence = sentence.lower()
    for key, word_array in end_json.items():
        for word in word_array:
            modified_sentence = modified_sentence.replace(word, key)  # Replace word with its corresponding value
    return modified_sentence
# Example usage
sentence = "What are the baseline study medications used in Sustain 6? What were the specific Diabetes and CV standards of care used in the study and the breakdown? What were the specific oral antidiabetic treatments used?"
modified_sentence = replace_words_with_dict(sentence,json_data)
print(modified_sentence)

what are the baseline study medications used in sustain 6? what were the specific diabetes and cardiovascular standards of care used in the study and the breakdown? what were the specific oral antidentificationiabetic treatments used?
